### create a dict of file name and api keys

In [ ]:
# Define base URL and parameters
base_url_profile = "https://api.census.gov/data/{year}/acs/acs5/profile?get=group({group})&for=tract:*&in=state:{state}"
base_url_subject = "https://api.census.gov/data/{year}/acs/acs5/subject?get=NAME,group({group})&for=tract:*&in=state:{state}"

years = list(range(2010, 2024))  # From 2010 to 2023
states = ["06", "32"]  # State codes as strings to preserve leading zeros
categories = {
    "DP03": "profile",
    "DP05": "profile",
    "S1501": "subject",
    "S2506": "subject"
}

data_links = {}

# Generate URLs using nested loops
for group, category in categories.items():
    for state in states:
        for year in years:
            key = f"{group}_{year}_{state}"
            if category == "profile":
                url = base_url_profile.format(year=year, group=group, state=state)
            else:
                url = base_url_subject.format(year=year, group=group, state=state)
            data_links[key] = url

# Print example of generated keys and links
for key, link in list(data_links.items())[:5]:
    print(f"{key}: {link}")  # Previewing first 5 links


### download and save files

In [ ]:
import os
import requests
import csv

# Define the save directory
save_directory = r"H:\GY350\File Downloads"

# Ensure the directory exists
os.makedirs(save_directory, exist_ok=True)

# Iterate over each key-value pair in the dictionary
for filename, url in data_links.items():
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()  # Convert response to JSON format
        
        # Define file path
        file_path = os.path.join(save_directory, f"{filename}.csv")
        
        # Write data to CSV file
        with open(file_path, mode='w', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerows(data)
        
        print(f"Successfully saved: {filename}.csv")
    else:
        print(f"Failed to download {filename}: {response.status_code}")

### finish downloading files

In [ ]:
import os
import requests
import csv

# Define the save directory
save_directory = r"H:\GY350\File Downloads"

# Ensure the directory exists
os.makedirs(save_directory, exist_ok=True)

# Identify files that are already downloaded
existing_files = set(f for f in os.listdir(save_directory) if f.endswith(".csv"))

# Define the starting point based on last downloaded file
start_downloading = False

# Iterate over remaining key-value pairs in the dictionary
for filename, url in data_links.items():
    expected_filename = f"{filename}.csv"
    
    # Start downloading from S1501_2023_06.csv onwards
    if expected_filename == "S1501_2023_06.csv":
        start_downloading = True
        continue  # Skip the last downloaded file itself
    
    if not start_downloading:
        continue  # Skip files until reaching the last downloaded file
    
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()  # Convert response to JSON format
        
        # Define file path
        file_path = os.path.join(save_directory, expected_filename)
        
        # Write data to CSV file
        with open(file_path, mode='w', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerows(data)
        
        print(f"Successfully saved: {expected_filename}")
    else:
        print(f"Failed to download {expected_filename}: {response.status_code}")

### rename files - run after all filed successfully saved

In [ ]:
import os

# Define the directory containing the downloaded files
directory = r"H:\GY350\File Downloads"

# Iterate through files in the directory
for filename in os.listdir(directory):
    if filename.endswith(".csv"):  # Ensure it's a CSV file
        parts = filename.rstrip(".csv").split("_")
        
        if len(parts) == 3:  # Ensure correct formatting
            new_filename = f"{parts[0]}_{parts[2]}_{parts[1]}.csv"
            old_path = os.path.join(directory, filename)
            new_path = os.path.join(directory, new_filename)
            
            os.rename(old_path, new_path)
            print(f"Renamed: {filename} -> {new_filename}")

### create new folder and copy files for manual edits

In [ ]:
import os
import shutil

# Define source and destination directories
source_directory = r"H:\GY350\File Downloads"
destination_directory = r"H:\GY350\CSV Manual Edits"

# Ensure the destination directory exists
os.makedirs(destination_directory, exist_ok=True)

# Copy all files from source to destination
for filename in os.listdir(source_directory):
    source_path = os.path.join(source_directory, filename)
    destination_path = os.path.join(destination_directory, filename)
    
    if os.path.isfile(source_path):  # Ensure it's a file
        shutil.copy2(source_path, destination_path)
        print(f"Copied: {filename}")

print("All files copied successfully.")


### cancelled due to lack of storage